# Segmenting and Clustering Neighborhoods in Toronto

In [37]:
from bs4 import BeautifulSoup
import pandas as pd #library for data analysis
import numpy as np #library to ahndle data in a vectorized manner
#import requests #library to handle requests

print("downloaded")

downloaded


### lets scrap and import data

In [38]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
main = requests.get(url).text
bsoup = BeautifulSoup(main, 'xml')

In [39]:
table=bsoup.find('table')

#breaking data into three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)


# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data



In [40]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [52]:
#lets remove row where Borough is "Not assigned"

df=df[df['Borough']!='Not assigned']
# give "Not assigned" Neighborhoods same name as Borough:
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)
# copy Borough value to Neighborhood if NaN:
df.Neighborhood.fillna(df.Borough, inplace=True)
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [55]:
new_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
new_df=new_df.reset_index(drop=False)
new_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [56]:
df_merge = pd.merge(df, new_df, on='Postalcode')
df_merge.drop(['Neighborhood'],axis=1,inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
